<a href="https://colab.research.google.com/github/jagrutimohanty/CMPE-297-Special-Topics-Advance-Deep-Learning/blob/main/Assignment3/Assignment_3_Part_5_297_News_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

     |████████████████████████████████| 3.1 MB 5.4 MB/s 
     |████████████████████████████████| 596 kB 49.7 MB/s 
     |████████████████████████████████| 3.3 MB 21.7 MB/s 
     |████████████████████████████████| 895 kB 48.6 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

In [3]:
dataset = fetch_20newsgroups(subset="all", shuffle=True, remove=("headers","footers", "quotes"))
target_names=dataset.target_names
news_text = dataset.data
labels = dataset.target
(train_texts,valid_texts,train_labels,valid_labels)=train_test_split(news_text, labels, test_size=0.3)

In [4]:
len(set(labels))

20

In [5]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=20)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
model.summary()



Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  15380     
Total params: 108,325,652
Trainable params: 108,325,652
Non-trainable params: 0
_________________________________________________________________


In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [8]:
train_texts[0:1]

['\n\tAs a matter of fact, I know it.\n\n\tThere were several shootings recently in which the guns had\nbeen purchased at the Baltimore Gunsmith, on Broadway - a favorite\nplace to get cheap "popular" pieces. By "popular" I am referring to\nthe kinds of guns our local youth gangs like: pistol grip shotguns,\ncheap magnums, and Tec-9s.\n\n\tI hate to poke a hole in your bubble, but I was referring\nto a specific gunstore, and specific incidents. Any other questions?']

In [9]:

train_encodings = tokenizer(train_texts, padding="max_length", truncation=True)
valid_encodings = tokenizer(valid_texts, padding="max_length", truncation=True)


In [14]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(valid_encodings),
    valid_labels
))

In [17]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
